In [1]:
import numpy as np
import yfinance as yf
from coskweness_cokurtosis import coskewness, cokurtosis
from portfolio_hubo_qaoa import HigherOrderPortfolioQAOA
import random
random.seed(5)

n_stocks = 3
stocks = ["AAPL", "GOOGL", "MSFT", "AMZN", "META", "TSLA", "WMT", "CAT", "KO", "IBM", "INTC", "CSCO", "ORCL", "QCOM", 
          "NVDA", "ADBE", "PYPL", "CRM", "ACN", "TXN", "AVGO", "INTU", "AMAT", "MU"]
stocks = random.sample(stocks, n_stocks)

data = yf.download(stocks, start="2021-01-01", end="2024-01-01")
prices_now = data["Close"].iloc[-1]
print(prices_now)
returns = data["Close"].pct_change().dropna()
stocks = returns.columns

numpy_returns = returns.to_numpy()
expected_returns = numpy_returns.mean(axis=0)*252
print(returns.mean())
print(expected_returns)
covariance_matrix = np.cov(numpy_returns, rowvar=False)*252
coskewness_tensor = coskewness(numpy_returns)#*(252**2)
cokurtosis_tensor = cokurtosis(numpy_returns)#*(252**3)

# Budget in dollars
# We compute the maximum number of shares we can buy for each stock
# At most n_qubits for each stock
n_qubits = 4
cheapest_stock = prices_now.min()
budget = n_qubits*cheapest_stock
print(budget)

risk_aversion = 3

portfolio_hubo = HigherOrderPortfolioQAOA(stocks=stocks,
                                          prices_now=prices_now,
                                          expected_returns=expected_returns, 
                                          covariance_matrix=covariance_matrix,
                                          budget=budget,
                                          coskewness_tensor=coskewness_tensor, 
                                          cokurtosis_tensor=cokurtosis_tensor,
                                          log_encoding = True, 
                                          layers = 1,
                                          risk_aversion = risk_aversion)

smallest_eigenvalues, smallest_bitstrings, first_excited_energy, optimized_portfolio, second_optimized_portfolio = portfolio_hubo.solve_exactly()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  3 of 3 completed

Ticker
KO      57.183517
MU      84.976990
TXN    164.521301
Name: 2023-12-29 00:00:00, dtype: float64
Ticker
KO     0.000321
MU     0.000551
TXN    0.000328
dtype: float64
[0.08078198 0.13882422 0.08254394]
228.73406982421875
Number of qubits for asset KO: 2
Number of qubits for asset MU: 2
Number of qubits for asset TXN: 1
Number of qubits per asset:  {'KO': 2, 'MU': 2, 'TXN': 1}
Constructing cost hubo with integer variables
Optimized Weights (considering variance, skewness and kurtosis):
KO: 29.56%
MU: 33.01%
TXN: 37.43%
Left over budget:  29.390045142318314
Optimized Discrete Allocation:
KO: 2
MU: 1
Maximized utility from classical higher moments:  -1.3392989409121916
Adding budget constraints to the cost function -> constructing full hubo problem
Replacing integer variables with binary variables
Simplifying the binary cost function
Converting binary cost function to Ising Hamiltonian
Constructing QAOA circuits
Performing warm start for QAOA with target bitstring  10010
Budget cons

In [2]:
print(smallest_eigenvalues)
print(smallest_bitstrings)
print(first_excited_energy)
print("Best: ", optimized_portfolio)
print("Second best: ", second_optimized_portfolio)
print("objective value 1: ", portfolio_hubo.get_objective_value(stocks, optimized_portfolio[0]))
print("objective value 2: ", portfolio_hubo.get_objective_value(stocks, second_optimized_portfolio[0]))

[np.float64(-17657081.33222471)]
[[1, 0, 0, 1, 0]]
-17641255.121456366
Best:  [{'KO': 1, 'MU': 2, 'TXN': 0}]
Second best:  [{'KO': 1, 'MU': 0, 'TXN': 1}]
objective value 1:  -12.046806687074923
objective value 2:  -1.6897773127855362


In [3]:
portfolio_hubo.solve_with_qaoa_jax()

Most probable state: 10111 and [[1, 0, 0, 1, 0]] with probs 10111
Most probable state: 01110 and [[1, 0, 0, 1, 0]] with probs 01110
Most probable state: 11111 and [[1, 0, 0, 1, 0]] with probs 11111
Most probable state: 11000 and [[1, 0, 0, 1, 0]] with probs 11000
Most probable state: 00001 and [[1, 0, 0, 1, 0]] with probs 00001
Most probable state: 10001 and [[1, 0, 0, 1, 0]] with probs 10001
Most probable state: 11010 and [[1, 0, 0, 1, 0]] with probs 11010
Most probable state: 00100 and [[1, 0, 0, 1, 0]] with probs 00100
Most probable state: 01000 and [[1, 0, 0, 1, 0]] with probs 01000
Most probable state: 00111 and [[1, 0, 0, 1, 0]] with probs 00111
Most probable state: 10000 and [[1, 0, 0, 1, 0]] with probs 10000
Most probable state: 00000 and [[1, 0, 0, 1, 0]] with probs 00000
Most probable state: 10111 and [[1, 0, 0, 1, 0]] with probs 10111
Most probable state: 01001 and [[1, 0, 0, 1, 0]] with probs 01001
Most probable state: 11001 and [[1, 0, 0, 1, 0]] with probs 11001
Most proba

(['00110', '10010'],
 Array(-9331709., dtype=float32),
 Array([[-0.03739724],
        [-0.01177937]], dtype=float32),
 770,
 [Array(0.0340483, dtype=float32), Array(0.03406329, dtype=float32)],
 [{'KO': 0, 'MU': 3, 'TXN': 0}, {'KO': 1, 'MU': 2, 'TXN': 0}])

In [4]:
two_most_probable_states, smallest_eigenvalue, params, total_steps, states_probs, optimized_portfolios = portfolio_hubo.solve_with_iterative_QAOA(max_layers=1)

Trying with 1 layers


c:\Users\valte\OneDrive - University of Helsinki\Desktop\quantum-portfolio\venv\lib\site-packages\pennylane\math\interface_utils.py:127: UserWarning: Contains tensors of types {'jax', 'autograd'}; dispatch will prioritize TensorFlow, PyTorch, and Jax over Autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


TypeError: einsum requires ndarray or scalar arguments, got <class 'autograd.numpy.numpy_boxes.ArrayBox'> at position 0.

In [ ]:
from portfolio_higher_moments_classical import HigherMomentPortfolioOptimizer


hef = HigherMomentPortfolioOptimizer(stocks= stocks,
                                     expected_returns=expected_returns, 
                                     covariance_matrix=covariance_matrix, 
                                     coskewness=coskewness_tensor, 
                                     cokurtosis=cokurtosis_tensor, 
                                     risk_aversion=risk_aversion)

weights = hef.optimize_portfolio_with_higher_moments()
print("Optimized weights:")
for asset, weight in zip(stocks, weights):
    print(f"{asset}: {weight:.2%}")
    
allocation, left_overs = hef.get_discrete_allocation(prices_now, budget)
print("Left over budget: ", left_overs)
print("Ratio of left over budget: ", left_overs/budget)

print("Optimized Discrete Allocation:")
for asset, amount in allocation.items():
    print(f"{asset}: {amount}")
    
value = portfolio_hubo.get_objective_value(stocks, allocation)
print("objective value: ", value)

Optimized weights:
KO: 29.56%
MU: 33.01%
TXN: 37.43%
Left over budget:  29.39006040110641
Ratio of left over budget:  0.12849006928651613
Optimized Discrete Allocation:
KO: 2
MU: 1
objective value:  -1.3393352129323297
